***Data Source1:  [Foursquare](https://developer.foursquare.com/places) API***

***Data Source2: [Yelp](https://www.yelp.com/developers/documentation/v3/get_started) API***

***Data Source3:  [CityBikes](https://citybik.es/) API***

In [1]:
# imports
import pandas as pd
import requests
import os
import time

# Foursquare

In [10]:
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')

***Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice.***

In [100]:
def fetch_foursquare(latitude, longitude, radius, api_key):
  
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius={radius}&fields=fsq_id%2Cname%2Clocation%2Ccategories%2Clink%2Cdescription%2Ctel%2Cfax%2Cemail%2Cwebsite%2Chours%2Chours_popular%2Crating%2Cstats%2Cpopularity%2Cprice%2Cmenu%2Cdate_closed%2Ctastes%2Cfeatures%2Cstore_id&sort=RATING"
    #url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&radius={radius}"
    
    headers = {"accept": "application/json",
               "Authorization": api_key}
    response = requests.get(url, headers=headers)
    return response


***Note:*** load the bike stations from 'calais_stations.csv'

In [2]:
bike_stations=pd.read_csv("../data/calais_stations.csv")
bike_stations

,station_id,station,latitude,longitude,no_of_bikes,network_id
0,116978dc6ed165d95a1ce386466bf165,a,50.946176,1.891176,0,vel-in
1,3b58c82f8aca0fd26a55dd5fcb82f9ec,b,50.960770,1.873922,0,vel-in
2,fedaa58e592a7e4156cf9f89e3f44560,01 : Lafayette,50.946609,1.858303,0,vel-in
3,a75dfb30a2a5e87b7ad50863923461ce,02 : Théatre,50.947377,1.853769,1,vel-in
4,9723c2436cfc9652882bf1b7c7e182e7,04 : Gare SNCF,50.953303,1.851278,2,vel-in
...,...,...,...,...,...,...
64,5f71f7dfa65b9d9262f2f966f3f481aa,COQUELLES,50.936090,1.800010,0,velectin
65,df6340be3e22aafbeb98c0f9a33567aa,UNIVERSITE,50.952317,1.879304,0,velectin
66,18885eecc620dfb8247e4b965eea7094,PLUVIOSE,50.964960,1.843470,0,velectin
67,0272a902cd9247078766c259f2ae3e1d,Gare SNCF,50.953262,1.850975,0,velectin


In [102]:
places=pd.DataFrame()
temp_counter=0
for index, row in bike_stations.iterrows():
    if temp_counter%20==0:
        time.sleep(2)
    temp_counter=temp_counter+1
    res = fetch_foursquare(latitude=row['latitude'], longitude=row['longitude'], radius=1000, api_key=FOURSQUARE_KEY) 
    df=pd.json_normalize(res.json()['results'])
    df['station']=row['station']
    places=pd.concat([places,df], ignore_index=True)
    
print("Process Completed")
places.head()
    

Process Completed


,fsq_id,categories,description,link,name,popularity,price,rating,tel,website,...,features.amenities.sit_down_dining,features.attributes.trendy,features.attributes.crowded,features.attributes.service_quality,store_id,features.attributes.healthy_diet,features.attributes.vegetarian_diet,features.attributes.groups_popular,features.attributes.dates_popular,features.attributes.families_popular
0,4dab2c0b0cb6a89c627db466,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",Commandez en ligne et retrouvez facilement tou...,/v3/places/4dab2c0b0cb6a89c627db466,McDonald's,0.985316,1.0,6.5,03 21 19 31 61,http://www.mcdonalds.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e4a0fe2eebce700081eee5d,"[{'id': 17069, 'name': 'Grocery Store', 'icon'...","Bienvenue dans votre magasin ! Ici, on répond ...",/v3/places/5e4a0fe2eebce700081eee5d,Intermarché SUPER Calais et Drive,0.469873,NaN,NaN,03 21 34 42 44,https://www.intermarche.com/magasin?pdvref=06940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1de3109bb15746cb265b50d6,"[{'id': 17035, 'name': 'Drugstore', 'icon': {'...",NaN,/v3/places/1de3109bb15746cb265b50d6,Pharmacie du Beau Marais,0.827848,NaN,NaN,03 21 97 18 30,http://www.pharmacie-du-beau-marais.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5dea4bd629e58b00086af5bd,"[{'id': 11000, 'name': 'Business and Professio...","Avec 25 années d'expérience, un professionnel ...",/v3/places/5dea4bd629e58b00086af5bd,JLM Renov'Batiments,0.536203,NaN,NaN,03 21 97 45 39,http://jlm-renovbatiments.fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cbb3734b7fe048b898863188,"[{'id': 12066, 'name': 'City Hall', 'icon': {'...",NaN,/v3/places/cbb3734b7fe048b898863188,Mairie Annexe,0.847595,NaN,NaN,03 21 46 90 20,http://www.calais.fr/fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
places.shape

(611, 87)

***Note:*** 
- We set a sleep time to 2 second to hault the query process after few queries as API have some sort of Rate Limit set.This timer is to make sure our process would not exceed the limit in any case.
- Save this DataFrame into csv file to avoid requering the API again

In [104]:
places.to_csv('../data/raw_places.csv', index=False)

In [61]:
#places=pd.read_csv('../data/raw_places.csv')


***Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)***

In [62]:
places.head()

,fsq_id,categories,description,link,name,popularity,price,rating,tel,website,...,features.amenities.sit_down_dining,features.attributes.trendy,features.attributes.crowded,features.attributes.service_quality,store_id,features.attributes.healthy_diet,features.attributes.vegetarian_diet,features.attributes.groups_popular,features.attributes.dates_popular,features.attributes.families_popular
0,4dab2c0b0cb6a89c627db466,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",Commandez en ligne et retrouvez facilement tou...,/v3/places/4dab2c0b0cb6a89c627db466,McDonald's,0.985316,1.0,6.5,03 21 19 31 61,http://www.mcdonalds.fr/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e4a0fe2eebce700081eee5d,"[{'id': 17069, 'name': 'Grocery Store', 'icon'...","Bienvenue dans votre magasin ! Ici, on répond ...",/v3/places/5e4a0fe2eebce700081eee5d,Intermarché SUPER Calais et Drive,0.469873,NaN,NaN,03 21 34 42 44,https://www.intermarche.com/magasin?pdvref=06940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1de3109bb15746cb265b50d6,"[{'id': 17035, 'name': 'Drugstore', 'icon': {'...",NaN,/v3/places/1de3109bb15746cb265b50d6,Pharmacie du Beau Marais,0.827848,NaN,NaN,03 21 97 18 30,http://www.pharmacie-du-beau-marais.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5dea4bd629e58b00086af5bd,"[{'id': 11000, 'name': 'Business and Professio...","Avec 25 années d'expérience, un professionnel ...",/v3/places/5dea4bd629e58b00086af5bd,JLM Renov'Batiments,0.536203,NaN,NaN,03 21 97 45 39,http://jlm-renovbatiments.fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cbb3734b7fe048b898863188,"[{'id': 12066, 'name': 'City Hall', 'icon': {'...",NaN,/v3/places/cbb3734b7fe048b898863188,Mairie Annexe,0.847595,NaN,NaN,03 21 46 90 20,http://www.calais.fr/fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
places.columns

Index(['fsq_id', 'categories', 'description', 'link', 'name', 'popularity',
       'price', 'rating', 'tel', 'website',
       'features.payment.credit_cards.accepts_credit_cards',
       'features.payment.credit_cards.amex',
       'features.payment.credit_cards.visa',
       'features.payment.credit_cards.master_card',
       'features.payment.digital_wallet.accepts_nfc',
       'features.food_and_drink.meals.breakfast',
       'features.food_and_drink.meals.lunch',
       'features.food_and_drink.meals.dessert',
       'features.food_and_drink.meals.dinner', 'features.services.delivery',
       'features.services.takeout', 'features.services.drive_through',
       'features.amenities.outdoor_seating', 'features.amenities.wifi',
       'hours.display', 'hours.is_local_holiday', 'hours.open_now',
       'hours.regular', 'location.address', 'location.admin_region',
       'location.country', 'location.cross_street',
       'location.formatted_address', 'location.locality', 'location.po

In [64]:
places.shape

(611, 87)

In [65]:
places.drop_duplicates(subset=places.columns.difference(['station']),inplace=True)

***Note:*** we exclude station column from the duplicates filter as this column is inserted by us therefore could contain diffrent value for duplicate records from the API

In [66]:
places.shape

(292, 87)

In [123]:
places['fsq_id'].value_counts(dropna=False)

4dab2c0b0cb6a89c627db466    1
53342eb8498e72792bbc993d    1
4f6caed1e4b01489b8e73815    1
53d37cfe498e2d92df6d6bca    1
5ac8cb4a598e643cd849eeb5    1
                           ..
010c7f316a70499524ba4707    1
ffba1e5374554ea5e4d71809    1
24e48e66fde44f9fadbf78d2    1
50db0bdbe4b047bb1f3ff23b    1
40db5e38acf24bcf6f8ca1cc    1
Name: fsq_id, Length: 292, dtype: int64

***Note:*** from the above query it seems that fsq_id is unique throuhout the data

In [124]:
places['store_id'].value_counts(dropna=False)

NaN      291
476.0      1
Name: store_id, dtype: int64

***Note:*** Almost all values of store_id are NULL so it is better to drop it

In [67]:
places.drop(['store_id'], axis=1,inplace=True)

In [68]:
places.shape

(292, 86)

In [69]:
fsq_category=places[['fsq_id','categories']]
fsq_category.head()

,fsq_id,categories
0,4dab2c0b0cb6a89c627db466,"[{'id': 13031, 'name': 'Burger Joint', 'icon':..."
1,5e4a0fe2eebce700081eee5d,"[{'id': 17069, 'name': 'Grocery Store', 'icon'..."
2,1de3109bb15746cb265b50d6,"[{'id': 17035, 'name': 'Drugstore', 'icon': {'..."
3,5dea4bd629e58b00086af5bd,"[{'id': 11000, 'name': 'Business and Professio..."
4,cbb3734b7fe048b898863188,"[{'id': 12066, 'name': 'City Hall', 'icon': {'..."


In [70]:
def dict_list_to_df(dff,col,id_col):
    rows = []
    for index, row in dff[col].iteritems():
        for item in list(eval((row))):
            item[id_col]=dff[id_col][index]
            rows.append(item)
    df = pd.DataFrame(rows)
    return df

fsq_category_df = dict_list_to_df(fsq_category,'categories','fsq_id')
fsq_category_df

,id,name,icon,fsq_id
0,13031,Burger Joint,{'prefix': 'https://ss3.4sqi.net/img/categorie...,4dab2c0b0cb6a89c627db466
1,13145,Fast Food Restaurant,{'prefix': 'https://ss3.4sqi.net/img/categorie...,4dab2c0b0cb6a89c627db466
2,17069,Grocery Store,{'prefix': 'https://ss3.4sqi.net/img/categorie...,5e4a0fe2eebce700081eee5d
3,17035,Drugstore,{'prefix': 'https://ss3.4sqi.net/img/categorie...,1de3109bb15746cb265b50d6
4,11000,Business and Professional Services,{'prefix': 'https://ss3.4sqi.net/img/categorie...,5dea4bd629e58b00086af5bd
...,...,...,...,...
339,12075,Post Office,{'prefix': 'https://ss3.4sqi.net/img/categorie...,4cd582129c214e41365034cb
340,11092,Landscaper and Gardener,{'prefix': 'https://ss3.4sqi.net/img/categorie...,11ea1c7943034897a654218e
341,17043,Clothing Store,{'prefix': 'https://ss3.4sqi.net/img/categorie...,ceb6a68aabb746e1a62fd8c7
342,15014,Hospital,{'prefix': 'https://ss3.4sqi.net/img/categorie...,c5c366344e0642af0d3f4997


In [71]:
fsq_category_df.drop(['icon'],axis=1,inplace=True)


In [72]:
fsq_category_df.duplicated(subset=['id','name','fsq_id']).value_counts(dropna=False)

False    344
dtype: int64

***Note:*** So now we have all the categories and corresponding 'fsq_id' in dataframe.We will save this in csv file for future use. from this dataset we can query the places and filter them using either category 'id' or 'name'. 'fsq_id' will work as a key to connect 'category' with the 'places' dataset

In [73]:
fsq_category_df.rename(columns={'id':'category_id','name':'category'},inplace=True)

In [74]:
fsq_category_df.to_csv('../data/fsq_category.csv', index=False)

***Note:*** We can drop the 'categories' column from 'places' dataframe

In [194]:
places.drop(['categories'],axis=1,inplace=True)

In [243]:
places['website'].fillna('Website not Found',inplace=True)

In [244]:
places['website'].value_counts(dropna=False)

Website not Found                              39
http://www.ville-marck.fr                       4
http://www.mcdonalds.fr/                        4
http://www.aldi.fr/                             3
http://www.mairie-calais.fr                     3
                                               ..
http://www.restaurantgalibot.com                1
http://www.chaussexpo.fr                        1
http://www.calais-vins.com                      1
https://www.afpa.fr/centre/centre-de-calais     1
http://www.lavieactive.fr                       1
Name: website, Length: 228, dtype: int64

In [228]:
places['rating'].fillna(0,inplace=True)

***Note:*** we can also for further in cleaning the places dataset like renaming columns , filling missing values , fixing data format etc. for now we save the whole data and filter it when needed accordingly

In [245]:
places.to_csv('../data/places.csv', index=False)

***Note:*** lets search for a category and find some features about the places. For the category id we can refer to the Foursquare API
 [Foursquare Category](https://location.foursquare.com/places/docs/categories)


In [82]:
places_df=pd.read_csv('../data/places.csv')

In [121]:
def search_places(category,features):
    temp_cat_df=fsq_category_df[(fsq_category_df['category_id']==category)|
                               (fsq_category_df['category'].str.contains(category.strip(),case=False)) ]
    search_result = pd.merge(temp_cat_df, places_df[features], on='fsq_id')
    return search_result
    

In [247]:
category_id=13003
features=['fsq_id','name','rating','tel','website','location.address'] #Don't forget to include fsq_id in feature list as it is a key to merge
search_result=search_places(category_id,features)
search_result

,category_id,category,fsq_id,name,rating,tel,website,location.address
0,13003,Bar,4c4964e53013a5937348ace2,Café des Dunes,0.0,03 21 34 14 17,Website not Found,87 route de Gravelines
1,13003,Bar,4e2860f6fa762797e59ca530,Le Calice,6.6,03 21 34 51 78,http://www.lecalice.com,55 boulevard Jacquard
2,13003,Bar,4c3f3acb15e8ef3b6962300e,Brasserie l'Hovercraft,6.8,03 21 34 59 73,Website not Found,11 place du Maréchal Foch
3,13003,Bar,51619390011c770ada84bc6c,Pop Rock Bar Calais,6.8,03 74 73 22 77,http://www.poprockbar.fr,59 place d'Armes
4,13003,Bar,4de284057d8b2547eb05ab5f,Les 3 Brasseurs,6.6,03 21 85 30 02,http://www.les3brasseurs.com,1001 boulevard du Kent
5,13003,Bar,50db0bdbe4b047bb1f3ff23b,Au Bowling de Calais,0.0,03 21 34 34 22,http://www.bowlingcalais.com,1 avenue Antoine de Saint-Exupéry
6,13003,Bar,6939f493cde14e50aae4c2e4,Le Galibot,0.0,03 66 98 10 41,http://www.restaurantgalibot.com,2 rue la Bruyère
7,13003,Bar,5ac8cb4a598e643cd849eeb5,La Marinerie,0.0,03 21 34 47 83,http://www.bouger-a.com,6 quai de la Colonne
8,13003,Bar,5a104a23b04056135fd543db,Café de la Gare,0.0,03 21 85 37 64,Website not Found,3140 rue Principale
9,13003,Bar,dadf746aae144afaf0c2cd73,Le Gambetta,0.0,03 21 34 79 72,Website not Found,60 boulevard Léon Gambetta


***Put your parsed results into a DataFrame***

***Note:*** Our result is in 'search_result' DataFrame

# Yelp

***Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.***

In [3]:
YELP_KEY = os.getenv('YELP_API_KEY')

In [7]:

def fetch_yelp(latitude, longitude, radius, api_key):
  
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&sort_by=rating&limit=20"
    headers = {"accept": "application/json",
              "Authorization": f"Bearer {api_key}" }
    response = requests.get(url, headers=headers)
    return response

In [8]:
yelp_places=pd.DataFrame()
temp_counter1=0
for index, row in bike_stations.iterrows():
    if temp_counter1%20==0:
        time.sleep(2)
    temp_counter1=temp_counter1+1
    res1 = fetch_yelp(latitude=row['latitude'], longitude=row['longitude'], radius=1000, api_key=YELP_KEY) 
    df1=pd.json_normalize(res1.json()['businesses'])
    df1['station']=row['station']
    yelp_places=pd.concat([yelp_places,df1], ignore_index=True)
    
print("Process Completed")
yelp_places.head()

Process Completed


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,station,price
0,o0AHPpZqqWc8jszVVNbooA,au-chien-chic-calais,Au Chien Chic,,False,https://www.yelp.com/biz/au-chien-chic-calais?...,1.0,"[{'alias': 'restaurants', 'title': 'Restaurant...",5.0,[],...,198 Boulevard Egalité,None,None,Calais,62100,FR,62,"[198 Boulevard Egalité, 62100 Calais, France]",a,NaN
1,WU6IaRGdEkYOm6r3QZOuUw,l-absinthe-calais-2,L'Absinthe,https://s3-media4.fl.yelpcdn.com/bphoto/1fVNlm...,False,https://www.yelp.com/biz/l-absinthe-calais-2?a...,2.0,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,[],...,12 rue la Paix,Bar,,Calais,62100,FR,62,"[12 rue la Paix, Bar, 62100 Calais, France]",a,NaN
2,8sdlNaz1z5PosLyL8qPMIA,campanile-calais,Campanile,https://s3-media2.fl.yelpcdn.com/bphoto/dXT2Io...,False,https://www.yelp.com/biz/campanile-calais?adju...,1.0,"[{'alias': 'french', 'title': 'French'}, {'ali...",2.0,[],...,35 Rue de Maubeuge - ZAC du Beau Marais,None,None,Calais,62100,FR,62,"[35 Rue de Maubeuge - ZAC du Beau Marais, 6210...",a,NaN
3,8gdWT3_-ssBF4KOfvuRSFA,calais-ferry-terminal-calais,Calais Ferry Terminal,https://s3-media3.fl.yelpcdn.com/bphoto/svPkAH...,False,https://www.yelp.com/biz/calais-ferry-terminal...,5.0,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]",3.5,[],...,Ave du Commandant Cousteau,,,Calais,62100,FR,62,"[Ave du Commandant Cousteau, 62100 Calais, Fra...",b,€€€€
4,NIGgI2mh1-Y4LBEzhF6Rew,la-buissonnière-calais-5,La Buissonnière,https://s3-media4.fl.yelpcdn.com/bphoto/EuvH1J...,False,https://www.yelp.com/biz/la-buissonni%C3%A8re-...,2.0,"[{'alias': 'french', 'title': 'French'}, {'ali...",5.0,[],...,10 Rue Neuve,None,None,Calais,62100,FR,62,"[10 Rue Neuve, 62100 Calais, France]",01 : Lafayette,NaN


In [9]:
yelp_places.shape

(612, 25)

In [49]:
yelp_places.to_csv('../data/raw_yelp_places.csv', index=False)

In [51]:
#yelp_places=pd.read_csv('../data/raw_yelp_places.csv')

In [11]:
yelp_places.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'phone', 'display_phone',
       'distance', 'coordinates.latitude', 'coordinates.longitude',
       'location.address1', 'location.address2', 'location.address3',
       'location.city', 'location.zip_code', 'location.country',
       'location.state', 'location.display_address', 'station', 'price'],
      dtype='object')

In [52]:
yelp_places.astype(str).duplicated(subset=yelp_places.columns.difference(['station'])).value_counts(dropna=False)

False    612
dtype: int64

In [53]:
yelp_places['id'].value_counts(dropna=False)

yETTFjY9EFBd-Y5BjHLv4g    19
Ya9NdWrIOdbOjb5Lc35h4w    16
NIGgI2mh1-Y4LBEzhF6Rew    16
m7fZizhIXQXjYs1TtqzsGA    16
NYXv3rwaSw_xN8vjGrVcZA    16
                          ..
kmkp-7qNDcp_UXWmL65X2A     1
sdsD9ecekgt-kStCwlWNrg     1
PFUWUmccqk_EJe0f7oP9aw     1
EpphVkcInag2AHpgQKTcIQ     1
KY-C3LzjBMJKuQKSzeNIlA     1
Name: id, Length: 99, dtype: int64

In [54]:
yelp_places['distance'].value_counts(dropna=False)

1007.120275    3
544.256790     3
813.456574     3
539.691928     3
843.146758     3
              ..
985.690534     1
1013.246181    1
1072.470157    1
1216.409423    1
726.801890     1
Name: distance, Length: 600, dtype: int64

In [55]:
yelp_places.astype(str).duplicated(subset=yelp_places.columns.difference(['station','distance'])).value_counts(dropna=False)

True     513
False     99
dtype: int64

***Note:*** Here we exclude 'station'and 'distance' from the duplicate check. 
- We choose 'station' for the same reason as in case of Foursquare API
- As the records fetch from the Yelp API could overlab corresponding to the same Latitude and Longitude. Therefore duplicate records could contains different diatance values.So we also exclude it.

In [56]:
yelp_places.drop_duplicates(subset=yelp_places.columns.difference(['station','distance']),inplace=True )


In [57]:
yelp_places.shape

(99, 25)

In [75]:
yelp_category=yelp_places[['id','categories']]
yelp_category.head()

,id,categories
0,o0AHPpZqqWc8jszVVNbooA,"[{'alias': 'restaurants', 'title': 'Restaurant..."
1,WU6IaRGdEkYOm6r3QZOuUw,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':..."
2,8sdlNaz1z5PosLyL8qPMIA,"[{'alias': 'french', 'title': 'French'}, {'ali..."
3,8gdWT3_-ssBF4KOfvuRSFA,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]"
4,NIGgI2mh1-Y4LBEzhF6Rew,"[{'alias': 'french', 'title': 'French'}, {'ali..."


In [76]:
yelp_category = dict_list_to_df(yelp_category,'categories','id')
yelp_category

,alias,title,id
0,restaurants,Restaurants,o0AHPpZqqWc8jszVVNbooA
1,beautysvc,Beauty & Spas,o0AHPpZqqWc8jszVVNbooA
2,bars,Bars,WU6IaRGdEkYOm6r3QZOuUw
3,danceclubs,Dance Clubs,WU6IaRGdEkYOm6r3QZOuUw
4,french,French,8sdlNaz1z5PosLyL8qPMIA
...,...,...,...
146,bistros,Bistros,12zcTN7VQkJJ1NfLlWybrQ
147,brasseries,Brasseries,12zcTN7VQkJJ1NfLlWybrQ
148,bbq,Barbeque,12zcTN7VQkJJ1NfLlWybrQ
149,brasseries,Brasseries,kEqfL9cBajOxhHL0OlqmWA


In [77]:
yelp_category.rename(columns={'alias':'category_alias','title':'category_title'},inplace=True)

In [78]:
yelp_category.to_csv('../data/yelp_category.csv', index=False)

In [107]:
yelp_category=pd.read_csv('../data/yelp_category.csv')

In [79]:
yelp_places.drop(['categories'],axis=1,inplace=True)

In [80]:
yelp_places['rating'].fillna(0,inplace=True)

In [81]:
yelp_places.to_csv('../data/yelp_places.csv', index=False)

***Note:*** Above process is similar to the process followed for Foursquare API 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [87]:
yelp_places_df=pd.read_csv('../data/yelp_places.csv')

In [88]:
yelp_places_df.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'rating', 'transactions', 'phone', 'display_phone', 'distance',
       'coordinates.latitude', 'coordinates.longitude', 'location.address1',
       'location.address2', 'location.address3', 'location.city',
       'location.zip_code', 'location.country', 'location.state',
       'location.display_address', 'station', 'price'],
      dtype='object')

In [89]:
def search_yelp_places(category,features):
    temp_cat_df1=yelp_category[(yelp_category['category_alias'].str.contains(category.strip(),case=False)) |
                               (yelp_category['category_title'].str.contains(category.strip(),case=False)) ]
    search_yelp_result = pd.merge(temp_cat_df1, yelp_places_df[features], on='id')
    return search_yelp_result

In [96]:
category1='bar'
features1=['id', 'alias', 'name', 'url', 'review_count','rating',  'phone', 
       'distance','location.city', 'location.zip_code', 'location.country',
       'location.state', 'location.display_address', 'station', 'price'] #Don't forget to include id in feature list as it is a key to merge
search_result1=search_yelp_places(category1,features1)
search_result1.head()

,category_alias,category_title,id,alias,name,url,review_count,rating,phone,distance,location.city,location.zip_code,location.country,location.state,location.display_address,station,price
0,bars,Bars,WU6IaRGdEkYOm6r3QZOuUw,l-absinthe-calais-2,L'Absinthe,https://www.yelp.com/biz/l-absinthe-calais-2?a...,2.0,4.0,3.366912e+10,311.803549,Calais,62100,FR,62,"['12 rue la Paix', 'Bar', '62100 Calais', 'Fra...",a,NaN
1,bars,Bars,h8vwDgh4ad6yShzDGD_adg,café-le-mirador-calais,Café Le Mirador,https://www.yelp.com/biz/caf%C3%A9-le-mirador-...,1.0,5.0,3.332134e+10,427.201137,Calais,62100,FR,62,"['47 Boulevard Jacquard', '62100 Calais', 'Fra...",01 : Lafayette,€€
2,bars,Bars,Nn2NxVA1hqh5qNUAlM_zIA,métropol-hôtel-calais-4,Métropol Hôtel,https://www.yelp.com/biz/m%C3%A9tropol-h%C3%B4...,2.0,4.0,3.332198e+10,1025.358533,Calais,62100,FR,62,"['45 quai du Rhin', '62100 Calais', 'France']",01 : Lafayette,NaN
3,bars,Bars,AitImzmkUvN_7jVVwXxKyA,le-channel-scène-nationale-de-calais-calais-4,Le Channel Scène Nationale de Calais,https://www.yelp.com/biz/le-channel-sc%C3%A8ne...,2.0,4.0,3.332147e+10,727.932240,Calais,62100,FR,62,"['173 Boulevard Gambetta', '62100 Calais', 'Fr...",01 : Lafayette,NaN
4,bars,Bars,cJidt90EUPJ-IaYODAlE-w,cafe-du-théâtre-calais-3,Cafe du Théâtre,https://www.yelp.com/biz/cafe-du-th%C3%A9%C3%A...,1.0,3.0,3.332136e+10,293.815071,Calais,62100,FR,62,"['9 Boulevard Pasteur', '62100 Calais', 'France']",01 : Lafayette,NaN


***Put your parsed results into a DataFrame***

***Note:*** Our result is in 'search_result1' DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

- No doubt both the APIs gives the similar type of information about a place
- But in Foursquare we have the flexibility to filter out only the required fields while initializing a request. Which helps alot and saves time and resources.


- If we see overall Foursquare API provides more no of records as compare to Yelp API

In [100]:
print('Foursquare places ',places_df.shape)
print('Yelp places ',yelp_places_df.shape)

Foursquare places  (292, 85)
Yelp places  (99, 24)


- Yelp provides only limited no of fields.On the other hand Foursquare provides vast range of fields including the 
more in-depth fields like chains,related_places etc

***But***

- Both Yelp and Foursquare provide the category based filter.But Yelp provide more flexible search using free hand string search on tha other hand
Foursquare need category_id for search as [Foursquare Category](https://location.foursquare.com/places/docs/categories)

In [105]:
yelp_places_df['rating'].value_counts() #Yelp

4.5    25
5.0    19
4.0    19
3.5    15
3.0    10
1.0     5
2.5     4
2.0     2
Name: rating, dtype: int64

In [106]:
places_df['rating'].value_counts() #Foursquare

0.0    243
6.3      6
6.4      6
6.5      5
7.8      3
6.8      3
6.0      3
6.6      3
7.7      3
7.2      2
7.1      2
8.0      2
8.1      2
7.0      2
6.7      2
7.6      1
7.9      1
5.9      1
5.6      1
8.4      1
Name: rating, dtype: int64

- from the above output we can see that most of the data in rating for Foursquare is 0.0. Although, Yelp provide less no of records but these records have valid ratings.

In [120]:
yelp_places_df['review_count'].value_counts()

1.0     35
2.0     18
3.0     13
6.0      5
5.0      4
4.0      4
18.0     4
11.0     3
10.0     2
9.0      2
8.0      2
12.0     2
14.0     1
19.0     1
7.0      1
13.0     1
25.0     1
Name: review_count, dtype: int64

- Foursquare API does not provide the review detail

***Result:*** From the above explanations it is clear that Yelp provide more reliable and complete data, despite the fact that no of records are less. 

***Get the top 10 restaurants according to their rating***

-Foursuare API

In [125]:
category='restaurant'
features=['fsq_id','name','rating','tel','website','location.address'] #Don't forget to include fsq_id in feature list as it is a key to merge
search_result=search_places(category,features)
search_result.loc[0:9].sort_values(by='rating',ascending=False)

,category_id,category,fsq_id,name,rating,tel,website,location.address
6,13338,Seafood Restaurant,527545e011d2215b96abcda9,Brasserie de la mer,7.8,03 21 96 17 72,http://www.brasseriedelamer.com,30 rue de la Mer
7,13148,French Restaurant,4c1635c277cea593e648d360,Le Centaure Brasserie,7.7,NaN,Website not Found,place d'Armes
8,13145,Fast Food Restaurant,4e0f5fdb8877936cb40ef389,Le Channel,7.2,03 21 34 42 30,http://www.restaurant-lechannel.com,3 boulevard de la Résistance
9,13148,French Restaurant,4e0f5fdb8877936cb40ef389,Le Channel,7.2,03 21 34 42 30,http://www.restaurant-lechannel.com,3 boulevard de la Résistance
3,13065,Restaurant,4e2860f6fa762797e59ca530,Le Calice,6.6,03 21 34 51 78,http://www.lecalice.com,55 boulevard Jacquard
0,13145,Fast Food Restaurant,4dab2c0b0cb6a89c627db466,McDonald's,6.5,03 21 19 31 61,http://www.mcdonalds.fr/,Mc Donald's Calais Mivoix avenue Guynemer
4,13145,Fast Food Restaurant,4c6d7424a437224b5a192db1,Subway,6.5,03 91 91 48 30,http://www.subwayfrance.fr/,102 boulevard Lafayette
1,13145,Fast Food Restaurant,4c8a038e1797236af7ae5d88,Le Saint Hubert,0.0,03 21 96 63 85,Website not Found,446 Grande rue du Petit Courgain
2,13148,French Restaurant,4c8a038e1797236af7ae5d88,Le Saint Hubert,0.0,03 21 96 63 85,Website not Found,446 Grande rue du Petit Courgain
5,13276,Sushi Restaurant,4ea1b9c9e5facb29c6f43c40,Oita Sushi,0.0,03 21 36 20 11,http://www.oita.fr,30 boulevard Léon Gambetta


- Yelp API

In [127]:
features1=['id', 'alias', 'name', 'url', 'review_count','rating',  'phone', 
       'distance','location.city', 'location.zip_code', 'location.country',
       'location.state', 'location.display_address', 'station', 'price'] #Don't forget to include id in feature list as it is a key to merge
search_result1=search_yelp_places(category,features1)
search_result1.loc[0:9].sort_values(by='rating',ascending=False)

,category_alias,category_title,id,alias,name,url,review_count,rating,phone,distance,location.city,location.zip_code,location.country,location.state,location.display_address,station,price
0,restaurants,Restaurants,o0AHPpZqqWc8jszVVNbooA,au-chien-chic-calais,Au Chien Chic,https://www.yelp.com/biz/au-chien-chic-calais?...,1.0,5.0,3.332198e+10,984.539669,Calais,62100,FR,62,"['198 Boulevard Egalité', '62100 Calais', 'Fra...",a,NaN
1,restaurants,Restaurants,2-h4Ywgr-DCGmXcdoVDuCg,chouchen-calais,chouchen,https://www.yelp.com/biz/chouchen-calais?adjus...,2.0,4.5,3.332136e+10,477.710304,Calais,62100,FR,62,"['40 Rue des Fontinettes', '62100 Calais', 'Fr...",01 : Lafayette,NaN
5,restaurants,Restaurants,2tiGTlBPBU700eCwS3ZQuQ,le-sesame-calais,Le Sesame,https://www.yelp.com/biz/le-sesame-calais?adju...,11.0,4.5,3.332197e+10,767.912627,Calais,62100,FR,62,"['27 Rue Mer', '62100 Calais', 'France']",04 : Gare SNCF,€€
6,restaurants,Restaurants,JSEpFIXiy4pg5kBgGI4fDQ,la-sole-meunière-calais-7,La Sole Meunière,https://www.yelp.com/biz/la-sole-meuni%C3%A8re...,6.0,4.5,3.332134e+10,830.347172,Calais,62100,FR,62,"['1 Boulevard de la Résistance', '62100 Calais...",04 : Gare SNCF,€€€€
7,restaurants,Restaurants,VXNVcBjRHaZuYaEV4fAe4A,le-channel-calais,Le Channel,https://www.yelp.com/biz/le-channel-calais?adj...,14.0,4.5,3.332134e+10,832.575866,Calais,62100,FR,62,"['3 boulevard de la Résistance', '62100 Calais...",04 : Gare SNCF,€€€€
8,restaurants,Restaurants,oZtAibjXjDUIUmYrnB53Hw,au-vieux-fourneau-calais,Au Vieux Fourneau,https://www.yelp.com/biz/au-vieux-fourneau-cal...,3.0,4.5,3.332135e+10,367.713249,Calais,62100,FR,62,"['4 Rue Royale', '62100 Calais', 'France']",04 : Gare SNCF,NaN
2,restaurants,Restaurants,pmuw4TBXX3rm-qCFrpbhcg,la-pataterie-calais-2,La Pataterie,https://www.yelp.com/biz/la-pataterie-calais-2...,1.0,4.0,3.332197e+10,163.334308,Calais,62100,FR,59,"['118 Boulevard Lafayette', '62100 Calais', 'F...",01 : Lafayette,NaN
3,restaurants,Restaurants,yiWBrqrlMyADaz6oq-8GlA,la-pataterie-calais,La Pataterie,https://www.yelp.com/biz/la-pataterie-calais?a...,1.0,4.0,3.332197e+10,177.673877,Calais,62100,FR,62,"['118 Boulevard La Fayette', '62100 Calais', '...",01 : Lafayette,NaN
9,restaurants,Restaurants,gnMBGzKub-RgoAIFwZrj2Q,flunch-coquelles,Flunch,https://www.yelp.com/biz/flunch-coquelles?adju...,3.0,3.5,3.332136e+10,1007.120275,Coquelles,62231,FR,62,"['Boulevard Kent', '62231 Coquelles', 'France']",12 : CC La Française,€€€
4,restaurants,Restaurants,cJidt90EUPJ-IaYODAlE-w,cafe-du-théâtre-calais-3,Cafe du Théâtre,https://www.yelp.com/biz/cafe-du-th%C3%A9%C3%A...,1.0,3.0,3.332136e+10,293.815071,Calais,62100,FR,62,"['9 Boulevard Pasteur', '62100 Calais', 'France']",01 : Lafayette,NaN
